In [23]:
import torch
from src.encoders import TextEncoder
from sentence_transformers.util import cos_sim

In [3]:
model = TextEncoder('sentence-transformers/all-mpnet-base-v2')

C:\Users\LorenzoBongiovanni\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LorenzoBongiovanni\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [30]:
texts = ['I went on the moon and I missed the sun a lot',
         'Once you go to mars you never go back',
         'it is raining a lot today',
         'I am soaked with rain']

def f():
    embeddings = model.encode(texts, no_grad=False)
    return cos_sim(embeddings, embeddings).sum(-1).mean()

In [34]:
x = f() 
print(x)

tensor(1.7266, grad_fn=<MeanBackward0>)


In [46]:
from src.score_propagation import UpwardScorePropagation

USP = UpwardScorePropagation({'a': 0.1, 'b':0.3}, {'a':0, 'b':1})
for p in USP.sigmoid_gate_model.parameters():
    p.requires_grad = False

In [49]:
from src.zero_shooter import TaxZeroShot
from src.dataset import WebOfScience
from globals import Paths


train_data = WebOfScience('train', topn=30)
tzs_model = TaxZeroShot(
    train_data.tax_tree,
    f'{Paths.SAVE_DIR}/label_alphas_WebOfScience.json',
    no_grad_zstc=False,
    no_grad_usp=True
)
list(tzs_model.encoder.encoder.model.parameters())


[Parameter containing:
 tensor([[ 0.0521, -0.0411,  0.0206,  ..., -0.0034, -0.1108, -0.0350],
         [ 0.0217,  0.0004, -0.0345,  ..., -0.0174,  0.0179, -0.0126],
         [ 0.0395, -0.0381, -0.0181,  ...,  0.0108, -0.0234, -0.0015],
         ...,
         [-0.0093,  0.0010,  0.0070,  ..., -0.0213, -0.0133,  0.0147],
         [ 0.0115, -0.0354,  0.0881,  ..., -0.1479, -0.0352, -0.0324],
         [ 0.0299,  0.0205,  0.0284,  ...,  0.0170, -0.0194, -0.0022]],
        requires_grad=True),
 Parameter containing:
 tensor([[-0.0015,  0.0054,  0.0038,  ...,  0.0039,  0.0031,  0.0009],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0202,  0.1353, -0.0309,  ...,  0.1075,  0.1267,  0.0025],
         ...,
         [-0.0384,  0.0312,  0.0344,  ...,  0.0119, -0.0297,  0.0089],
         [-0.0337,  0.0046,  0.0687,  ...,  0.0239, -0.0486, -0.0136],
         [-0.1274,  0.0492, -0.0992,  ...,  0.1985,  0.3918,  0.0317]],
        requires_grad=True),
 Parameter con